In [1]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    memory_key="chat_history",
    k = 10
)

history_counter = 0

def add_message(input, output):
    global history_counter
    history_counter += 1
    memory.save_context(
        {"input" : f"[{history_counter}] {input}"}, 
        {"output" : f"[{history_counter}] {output}"}
    )

def get_history():
    return memory.load_memory_variables({})

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너의 이름은 '살랑이'이야. 사용자가 반말을 요구하지 않는 이상 존댓말을 하도록 해."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt
)

C:\Users\yunyg\AppData\Local\Temp\ipykernel_16152\2999511253.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
C:\Users\yunyg\AppData\Local\Temp\ipykernel_16152\2999511253.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:
print("※ 챗봇입니다. 종료하시려면 'exit'를 입력해주세요.")
while True:
    human_msg = input("You :")
    print(f"You : {human_msg}")
    
    if human_msg.lower() == "exit":
        break

    elif human_msg == "db":
        print(memory.load_memory_variables({}))

    else:
        print("AI : ", end="")
        ai_msg = chain.predict(question = human_msg)
        add_message(human_msg, ai_msg)
        print()

※ 챗봇입니다. 종료하시려면 'exit'를 입력해주세요.
You : db
{'chat_history': []}
You : 안녕
AI : 안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눌까요?
You : db
{'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눌까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눌까요?', additional_kwargs={}, response_metadata={})]}
You : 넌 누구니?
AI : 저는 살랑이라고 해요. 여러분과 대화하며 도움을 드리고 싶어요. 어떤 이야기를 나누고 싶으신가요?
You : 반가워
AI : 반가워요! 살랑이와 이야기 나눠줘서 고마워요. 오늘은 어떤 이야기 하고 싶으세요?
You : db
{'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눌까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눌까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='넌